In [ ]:
!pip -q install langchain huggingface_hub openai tiktoken
!pip -q install chromadb duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.8 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.250
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# LangChain Expression Language


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from langchain.schema.output_parser import StrOutputParser

In [ ]:
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
    )

model2 = OpenAI(
    model="text-davinci-003",
    temperature=0
    )

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me an intersting fact about {subject}"
    )

In [ ]:
chain = prompt | model

In [ ]:
chain.invoke({"subject": "Elvis"})

AIMessage(content='One interesting fact about Elvis Presley is that he was a black belt in karate. He began studying martial arts in the 1950s and eventually earned his black belt in 1960. Elvis was passionate about karate and even incorporated some of the moves into his performances. He often practiced with his friends and bodyguards, and even had a custom-made karate uniform with his name embroidered on it.', additional_kwargs={}, example=False)

In [ ]:
chain = prompt | model | StrOutputParser()

In [ ]:
chain.invoke({"subject": "Elvis"})

'One interesting fact about Elvis Presley is that he was a black belt in karate. He began studying martial arts in the 1950s and eventually earned his black belt in 1960. Elvis was passionate about karate and even incorporated some of the moves into his stage performances. He often practiced martial arts as a way to stay fit and maintain discipline in his life.'

In [ ]:
chain = prompt | model2 | StrOutputParser()

In [ ]:
chain.invoke({"subject": "Elvis"})

'\n\nElvis Presley was the first rock and roll artist to be inducted into the Country Music Hall of Fame.'

## Bindings

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me 3 intersting facts about {subject}"
    )

In [ ]:
chain = prompt | model.bind(stop=["\n"]) | StrOutputParser()


In [ ]:
chain.invoke({"subject": "Elvis"})

'1. Elvis Presley, often referred to as the "King of Rock and Roll," was born on January 8, 1935, in Tupelo, Mississippi. He began his music career in the mid-1950s and quickly rose to fame with his unique blend of rockabilly, country, and rhythm and blues.'

## Adding OpenAI Functions

In [ ]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]
functions_chain = prompt | model.bind(function_call= {"name": "joke"}, functions= functions)

In [ ]:
functions_chain.invoke({"subject": "bears"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'joke', 'arguments': '{\n  "setup": "Why don\'t bears wear shoes?",\n  "punchline": "Because they have bear feet!"\n}'}}, example=False)

### Functions Output Parser


In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

functions_chain = (
    prompt
    | model.bind(function_call= {"name": "joke"}, functions= functions)
    | JsonOutputFunctionsParser()
)

In [ ]:
response = functions_chain.invoke({"subject": "bears"})

response

{'setup': "Why don't bears wear shoes?",
 'punchline': 'Because they have bear feet!'}

In [ ]:
response['punchline']

'Because they have bear feet!'

In [ ]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

functions_chain = (
    prompt
    | model.bind(function_call= {"name": "joke"}, functions= functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [ ]:
functions_chain.invoke({"subject": "bears"})

"Why don't bears wear shoes?"

## Retrievers

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
# Create the retriever
fake_docs = ["James bond works for MI6","Bond is a spy",
             "James Bond has a licence to kill", "James Bond likes cats"]
vectorstore = Chroma.from_texts(fake_docs, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("Who is James Bond?")

'James Bond is a spy who works for MI6.'

In [ ]:
chain.invoke("What does James Bond like to do?")

'Based on the given context, it can be inferred that James Bond likes cats.'

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question"),
    "language": itemgetter("language")
} | prompt | model | StrOutputParser()

In [ ]:
chain.invoke({"question": "where does James work?", "language": "english"})

'James works for MI6.'

In [ ]:
chain.invoke({"question": "where does James work?", "language": "italian"})

'James lavora per MI6.'

## Tools

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

In [ ]:
search = DuckDuckGoSearchRun()

In [ ]:
template = """turn the following user input into a search query for a search engine:

{input}"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = prompt | model | StrOutputParser() | search

In [ ]:
chain.invoke({"input": "Who played james bond first"})

'As of 2020, there have been seven actors who have played the iconic role of James Bond: Sean Connery, David Niven, George Lazenby, Roger Moore, Timothy Dalton, Pierce Brosnan and Daniel Craig. Connery is the actor who has played Bond the most times, with a total of six films under his belt. This includes the first ever Bond film, "Dr. Sir Roger George Moore KBE (14 October 1927 - 23 May 2017) was an English actor. He was the third actor to portray fictional secret agent James Bond in the Eon Productions/MGM Studios film series, playing the character in seven feature films between 1973 and 1985. Moore\'s seven appearances as Bond, from Live and Let Die to A View to a Kill, are the most of any actor in the Eon-produced entries.'

In [ ]:
chain = prompt | model | StrOutputParser()
chain.invoke({"input": "Who played james bond last"})

"On this list of shortest and tallest James Bond actors, Seas is certainly not the shortest. 2. David Niven (1967), Height - 5 feet 11 ¼ (155 cm) David Niven ( Image Source) David Niven was said to have been Fleming's choice for the role of James Bond before Connery took up the mantle. The actor, who had attended Sandhurst Military Academy ..."

## Arbitary Functions

In [ ]:
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain1 = prompt | model

chain = {
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
} | prompt | model

In [ ]:
chain.invoke({"foo": "bars", "bar": "gahs"})

AIMessage(content='4 + 16 equals 20.', additional_kwargs={}, example=False)